# Factory Simulation with SimPy
## Scenario
A factory gets **raw material**. The raw material is processed/assembled into pieces by **Piece Assembly Machines**. Then, the pieces are assembled into models by the **Model Assembly Machines**. From time to time, the machines break.

When a machine breaks, the factory calls a repair center, which has other callers. When the factory's request reaches an operator, they decide if intervention is necessary. If no intervention is needed, the factory workers can repair the machine by themselves. If intervention is necessary, a repairman is called to the premises, who then repairs the machine.

### Math Reminders - Exponential & Lognormal Distributions
They can be found in `pharmacy_sim.ipynb`. 

### Math Reminder - Normal Distribution
Also called the **Gaussian Distribution** is one of the most commonly used distributions in statistics.

Its PDF is:

$$
f(x) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp(-\frac{(x-\mu)^2}{2\sigma^2})
$$